INTRODUÇÃO
Explicar objetivo do trabalho
Apresentar banco de dados
Definição de Target e Feature

MINERAÇÃO DE DADOS
Limpeza do dataset

In [ ]:
# Código da mineração de dados

CARACTERÍSTICAS DO DATASET FINAL
Descrever cada coluna e categorias (se houver) das colunas escolhidas para serem mantidas e usadas no dataset

TECNOLOGIAS USADAS
Bibliotecas e funções utilizadas para a manutenção e elaboração dos estudos sobre o dataset

MODELO DE PREDIÇÃO - TREINAMENTO


CARACTERÍSTICAS DO DATASET FINAL
Descrever cada coluna e categorias (se houver) das colunas escolhidas para serem mantidas e usadas no dataset

TECNOLOGIAS USADAS
Bibliotecas e funções utilizadas para a manutenção e elaboração dos estudos sobre o dataset

MODELO DE PREDIÇÃO - TREINAMENTO


In [ ]:
# código do modelo de treinamento

MODELO DE PREDIÇÃO - RESULTADOS

In [ ]:
# código do modelo sendo aplicado no teste

VALIDAÇÃO DOS RESULTADOS

In [ ]:
# código de validação de resultados 